# Intro
Есть несколько вариантов представлений текстов - с помощью OneHotEncoder, CountVectorizer, TfidfVectorizer. Все они работают по общему принципу: мы как-то переводим тексты в вектора, а с векторами мы уже хорошо умеем работать.


Все эти подходы основываются на частоте появления слов в нашем датасете (коллекции документов). Они хорошо работают во многих задачах, но были придуманы в 50ых - 70ых годах прошлого века. Огромным недостатком применения этих методов на чистых необработанных данных является их скорость - на больших данных вы просто не сможете их обучить с плохим аппаратным обеспечением. В этом ноутбуке рассмотрим несколько методов, как оптимизовать обработку таких данных, чтобы минимизировать потерю качества и максимизировать скорочть обработки. А именно рассмотрим:
* уменьшение размерности с помощью PCA
* модуль fasttext от Facebook, которая делает эмбеддинги

Чтобы запустить этот ноутбук вам понадобится:

- скачать и распаковать архив crawl-300d-2M.vec.zip с сайта https://fasttext.cc/docs/en/english-vectors.html (либо просто скопировать мой ноутбук - тут уже загружен этот файл)

- установить gensim с помощью `pip install gensim`

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time


Рассмотрим простой пример - задачу небинарной классификации текстов из ноутбука по NLP:

In [ ]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset = 'train', categories = categories, shuffle = True, random_state = 42)
twenty_test = fetch_20newsgroups(subset='test', categories = categories,shuffle = True, random_state = 42)
X_train = twenty_train.data
y_train = twenty_train.target
X_test = twenty_test.data
y_test = twenty_test.target

В этот раз загрузим весь словарь для улучшения качества наших эмбеддингов. Если он у вас не влезает в память, загрузите часть словаря.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)
logreg = LogisticRegression(solver = 'liblinear', multi_class = 'ovr', random_state = 1)

logreg.fit(X_train_tfidf, y_train)
y_pred = logreg.predict(X_test_tfidf)
print(accuracy_score(y_test, y_pred))

OutPut:  0.8868175765645806
print(X_train_tfidf.shape)


Простое использование `TfidfVectorizer + LogisticRegression` дает нам приличную accuracy 0.896. Но будь у нас больше данных у нас могли бы возникнуть проблемы при обучении из-за размера матрицы `X_train_tfidf` - сейчас он всего `(2257, 35788)`.

Логичный способ уменьшить размер матрицы - применить  методы понижения размерности, например PCA. В общем случае это может быть непросто, т.к. матрицы получаемые с помощью `TfidfVectorizer` бывают очень большие и почти всегда очень разреженные. Но здесь у нас специально взят очень маленький датасет, так что мы легко можем воспользоваться PCA.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 300, random_state = 1)
X_train_tfidf_pca = pca.fit_transform(X_train_tfidf.todense())
X_test_tfidf_pca = pca.transform(X_test_tfidf.todense())

logreg.fit(X_train_tfidf_pca, y_train)
y_pred_pca = logreg.predict(X_test_tfidf_pca)
print(accuracy_score(y_test, y_pred_pca))

При понижении размерности с `35788` до `300` качество упало, но не очень сильно.

Давайте теперь рассмотрим подход, основанный на эмбеддингах из fastText.

Word2Vec - это нейросетевая модель, более подробно она и другие похожие модели рассматриваются в курсе по DL. Но результатом работы данной модели является словарь эмбеддингов, позволяющий сопоставлять словам их векторные представления. Чтобы пользоваться такими словарями нам не обязательно прямо сейчас понимать, как именно работает эта модель.


За время прошедшее с публикации Word2Vec появилось много аналогичных алгоритмов, а также готовые словари с миллионами слов. Они получены из моделей качественно обученных на больших датасетах. Пример таких словарей - fastText от Facebook, он есть в публичном доступе для 157 языков.


Появились также и библиотеки, позволяющие обучить свою модель Word2Vec на своей коллекции документов в пару строк кода. Мы рассмотрим библиотеку gensim.


В большинстве случаев, самостоятельно обученная модель будет уступать готовым, но есть случаи, когда выгодно обучить модель самому - например если вы работаете с задачей в какой-то особенной предметной области и ваши тексты сильно отличаются от стандартных текстов для обучения моделей.

Попробуем считать вектор документа как среднее арифметическое векторов его слов. Слова, которых нет в нашем словаре, будем заменять на средний вектор по всему словарю.

Давайте загрузим словарь эмбеддингов FastText для английского языка (первые 2000000 слов). Код функции `load_vectors` взят с сайта fastText и немного модифицирован.

In [ ]:
import io
from tqdm import tqdm
from itertools import islice
import numpy as np

def load_vectors(fname, limit):
  fin = io.open(fname, 'r', encoding = 'utf-8', newline = '\n', errors = 'ignore')
  n, d = map(int, fin.readline().split())
  data = {}
  for line in tqdm(islice(fin, limit), total = limit):
    tokens = line.rstrip().split(' ')
    data[tokens[0]] = np.array(list(map(float, tokens[1:])))
  return data

Загрузим весь словарь для улучшения качества наших эмбеддингов. У меня следующая строчка кода загружалась примерно 4 минуты. Если он у вас не влезает в память, загрузите часть словаря (уменьшите второй параметр).

In [ ]:
vecs = load_vectors('../input/crawluy/crawl-300d-2M.vec', 2000000)   

Теперь в переменной `vecs` у нас обычный питоновский `dict`, в котором ключи - это слова, а значения - вектора в 300-мерном пространстве.

In [ ]:
import numpy as np

zero = sum(vecs.values()) / len(vecs)
def text2vec(text):
  words = text.split()
  return sum(list(map(lambda w: np.array(list(vecs.get(w, zero))), words))) / len(words)

Преобразуем `X_train` и `X_test` описанным выше способом.



In [ ]:
X_train_vec = list(map(lambda text: text2vec(text), X_train))
X_test_vec = list(map(lambda text: text2vec(text), X_test))

In [ ]:
logreg.fit(X_train_vec, y_train)
y_pred_vec = logreg.predict(X_test_vec)
print(accuracy_score(y_test, y_pred_vec))

Как мы видим, качество значительно упало. Это связано с тем, что усреднять слова в документе - не лучший способ получить его векторное представление. Есть много слов, которые встречаются часто и есть почти во всех документах. При таком способе подсчета они будут делать вектора документов слишком похожими друг на друга.

Значит, нам нужно давать словам разные веса, когда мы вычисляем вектор документа. Большой вес должен быть у слов, которые часто встречаются в этом документе, но редко в других. Но это как раз tf-idf. (Также можно использовать просто idf. TF - частота слова в документе - итак будет учтена, т.к. мы суммируем вектора слов в документе).


Осталось понять, как посчитать такую взвешенную сумму векторов. Для этого можно составить матрицу из векторов слов, встречающихся в нашей коллекции (они обязательно должны идти в том же порядке, что и в матрице из `TfidfVectorizer` - но там слова будут столбцами, а тут - строками), после чего перемножить 2 матрицы.


Пусть

- DOCS - число документов в коллекции

- WORDS - число уникальных слов

- DIM - размерность пространства эмбеддингов из Word2Vec


Тогда первая матрица будет размера `(DOCS, WORDS)`, вторая - `(WORDS, DIM)`, а их произведение - `(DOCS, DIM)` - это и будет матрица векторных представлений для каждого документа в коллекции.

In [ ]:
dim = 300
vocab = np.zeros((len(tfidf.vocabulary_.keys()), dim))
for key in tqdm(tfidf.vocabulary_.keys()):
  vocab[tfidf.vocabulary_[key]] = vecs.get(key, zero)

In [ ]:
X_train_weighted = X_train_tfidf.dot(vocab)
X_test_weighted = X_test_tfidf.dot(vocab)

In [ ]:
logreg.fit(X_train_weighted, y_train)
y_pred_weighted = logreg.predict(X_test_weighted)
print(accuracy_score(y_test, y_pred_weighted))


# Заключение
В итоге мы понизили размерность матрицы из `TfidfVectorizer` не используя PCA или другие методы понижения размерности (которые часто плохо работают в задачах с текстами) и при этом получили качество выше, чем с использованием PCA и совсем немного ниже, чем при использовании исходной матрицы. В вашем соревновании лучше использовать X_train_weighted, а не X_train_vec, последний будет загружаться слишком долго, а мы ведь уже поняли, что первый даст результат лучше.

## Fun facts
Давайте в vecs загрузим словарь поменьше

In [ ]:
vecs = load_vectors('../input/crawluy/crawl-300d-2M.vec', 100000) 

Теперь в переменной `vecs` у нас обычный питоновский `dict`, в котором ключи - это слова, а значения - вектора в 300-мерном пространстве.


Это пространство хорошо тем, что в нем похожим по значению словам соответствуют похожие вектора - т.е. близкие друг к другу точки в 300-мерном пространстве.


Напишем простую функцию `get_k_nearest_neighbors`, которая будет для данного вектора находить ближайшие к нему вектора в словаре.

In [ ]:
def get_k_nearest_neighbors(vec, k):
  return list(zip(*sorted(list(map(lambda key: (np.linalg.norm(vec - vecs[key]), key), vecs.keys())))))[1][:k]

print(get_k_nearest_neighbors(vecs['Paris'], 20))
print(get_k_nearest_neighbors(vecs['brother'], 20))

Как видно из примера это условие выполняется - наиболее близкими к слову `Paris` оказались слова связанные с Францией и названия городов, в основном французских. Наиболее близкими к слову `brother` оказались слова, обозначающие родственников.


У эмбеддингов, получаемых с помощью Word2Vec есть еще одна интересная особенность - если мы можем сказать, что `А относится к B, как С к D`, то вектора слов `A - B` и `C - D` будут довольно похожи. Это значит, что если мы рассмотрим вектор `A - B + D`, то вектор `C` часто будет оказываться среди его ближайших соседей:

In [ ]:
get_k_nearest_neighbors(vecs['Paris'] - vecs['France'] + vecs['Germany'], 1)

In [ ]:
get_k_nearest_neighbors(vecs['brother'] - vecs['man'] + vecs['woman'], 1)

Самым известным примером таких соотношений векторов является равенство

`king - man + woman ≈ queen`. В данном словаре оно не совсем точно выполняется:

In [ ]:
get_k_nearest_neighbors(vecs['king'] - vecs['man'] + vecs['woman'], 5)